In [34]:
import requests
import json
import numpy as np
import pandas as pd
import datetime as dt
from shroomdk import ShroomDK
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## 0. Setting

In [3]:
#API Setting
df_api = pd.read_fwf('API_KEY.txt', colspecs= 'infer')
API_KEY = df_api.columns[0]
sdk = ShroomDK(API_KEY )

In [95]:
# SQL Scripts

#Pool Info
pool_info = """

SELECT
	POOL_NAME,
  	POOL_ADDRESS,
  	BLOCK_TIMESTAMP::DATE AS CREATION_DATE,
  	TOKEN0_ADDRESS,
  	TOKEN0_SYMBOL,
  	TOKEN1_ADDRESS,
  	TOKEN1_SYMBOL,
  	FEE_PERCENT
FROM
	ethereum.uniswapv3.ez_pools
WHERE BLOCKCHAIN = 'ethereum'
ORDER BY BLOCK_TIMESTAMP ASC
-- LIMIT 1000
"""

#Swap Volume
#Fees colected from trades
swap_vol = """
WITH  SWAP_DAILY AS (
  SELECT
  		DATE_TRUNC('day',BLOCK_TIMESTAMP) AS DATE,
  		TX_HASH,
    	POOL_ADDRESS,
    	POOL_NAME,
    	RECIPIENT,
    	TOKEN0_SYMBOL
    	AMOUNT0_USD,
    	TOKEN1_SYMBOL,
    	AMOUNT1_USD,
    	(ABS(AMOUNT0_USD) + ABS(AMOUNT1_USD))/2 AS VOL_USD
    
  FROM
  	ethereum.uniswapv3.ez_swaps
  WHERE (AMOUNT0_USD > 0 OR AMOUNT1_USD > 0)
    AND BLOCKCHAIN = 'ethereum'
  	AND DATE >= CURRENT_DATE - INTERVAL '6months'
  ORDER BY 1 ASC),

  
AGG_TABLE AS (
SELECT
  DATE,
  POOL_ADDRESS,
  POOL_NAME,
  COUNT(TX_HASH) AS SWAP_COUNT,
  SUM(VOL_USD) AS AMOUNT_USD

FROM
	SWAP_DAILY
GROUP BY 1,2,3	
ORDER BY 1)

SELECT
  A.DATE,
  --A.POOL_ADDRESS,
  A.POOL_NAME,
  SWAP_COUNT,
  AMOUNT_USD,
  AMOUNT_USD * (FEE_PERCENT/100) AS  FEE_AMOUNT_USD

FROM
	AGG_TABLE A
	LEFT JOIN ethereum.uniswapv3.ez_pools P  ON A.POOL_ADDRESS = P.POOL_ADDRESS
	
--LIMIT 1000
"""

#TVL Growth

tvl_growth = """
WITH L_PROVIDERS AS (
	SELECT
  		LIQUIDITY_PROVIDER,
  		ACTION,
  		SUM(COALESCE(AMOUNT0_USD,0)+ COALESCE(AMOUNT1_USD,0)) AS LIQUIDITY_AMOUNT
  	FROM
		ethereum.uniswapv3.ez_lp_actions
	
	WHERE (AMOUNT1_USD > 0 OR AMOUNT0_USD > 0)
  		--AND DATE_TRUNC('day', BLOCK_TIMESTAMP) >= CURRENT_DATE - INTERVAL '12months'
  	GROUP BY 1,2
  	ORDER BY 1),

TVL_GROWTH AS 
  (SELECT
	DATE_TRUNC('day',BLOCK_TIMESTAMP) AS DATE,
  	POOL_ADDRESS,
  	POOL_NAME,
  	ACTION,
	AMOUNT0_USD,
  	AMOUNT1_USD,
  	COALESCE(AMOUNT0_USD,0)+ COALESCE(AMOUNT1_USD,0) AS LIQUIDITY_USD,
  	(CASE
  		WHEN ACTION = 'DECREASE_LIQUIDITY' THEN -LIQUIDITY_USD
  		WHEN ACTION = 'INCREASE_LIQUIDITY' THEN LIQUIDITY_USD
  		ELSE LIQUIDITY_USD END) AS LIQUIDITY_USD_ADJ,
  	LIQUIDITY_ADJUSTED,
  	LIQUIDITY_PROVIDER,
  	(CASE
  		WHEN LIQUIDITY_PROVIDER IN (SELECT LIQUIDITY_PROVIDER FROM L_PROVIDERS WHERE LIQUIDITY_AMOUNT >= 100000) THEN 'Whale'
  		ELSE 'Common User' END) AS L_PROVIDER_TYPE  	
FROM
	ethereum.uniswapv3.ez_lp_actions
WHERE (AMOUNT1_USD > 0 OR AMOUNT0_USD > 0)
  	AND DATE >= CURRENT_DATE - INTERVAL '6months'
ORDER BY DATE ASC)
  
SELECT
  DATE,
  POOL_NAME,
  L_PROVIDER_TYPE,
  SUM(LIQUIDITY_USD_ADJ) AS NET_AMOUNT_USD,
  COUNT(DISTINCT LIQUIDITY_PROVIDER) AS UNIQUE_L_PROVIDERS

FROM
	TVL_GROWTH
GROUP BY 1,2,3
ORDER BY 1,2,3
--LIMIT 1000
"""

tvl_overtime = """
WITH STATS AS (
  SELECT
    	DATE_TRUNC('day',BLOCK_TIMESTAMP) AS DATE,
    	POOL_ADDRESS,
    	POOL_NAME,
  		TOKEN0_BALANCE_ADJUSTED,
  		TOKEN1_BALANCE_ADJUSTED,
    	(TOKEN0_BALANCE_USD + TOKEN1_BALANCE_USD) AS TVL_USD,
    	PRICE_0_1,
    	PRICE_1_0,
    	ROW_NUMBER() OVER(PARTITION BY POOL_ADDRESS ORDER BY DATE DESC) AS ROW_NUMBER
  
  FROM
  	ethereum.uniswapv3.ez_pool_stats
  
  WHERE
  	DATE >= CURRENT_DATE - INTERVAL '1 day'
    	AND DATE < CURRENT_DATE
  
  ORDER BY 1,2, ROW_NUMBER DESC)

SELECT *
FROM
	STATS
WHERE ROW_NUMBER = 1
--LIMIT 10000 
"""

In [ ]:
# Function to retrieve data from more than one page (100K rows)
def querying_pagination(query_string):
    sdk = ShroomDK(API_KEY)
    
    # Query results page by page and saves the results in a list
    # If nothing is returned then just stop the loop and start adding the data to the dataframe
    result_list = []
    for i in range(1,11): # max is a million rows @ 100k per page
        data=sdk.query(query_string,page_size=100000,page_number=i)
        if data.run_stats.record_count == 0:  
            break
        else:
            result_list.append(data.records)
        
    # Loops through the returned results and adds into a pandas dataframe
    result_df=pd.DataFrame()
    for idx, each_list in enumerate(result_list):
        if idx == 0:
            result_df=pd.json_normalize(each_list)
        else:
            result_df=pd.concat([result_df, pd.json_normalize(each_list)])

    return result_df

In [59]:
# Function to convert dataframe to google sheet
def df_to_sheet(df,sheet_name, worksheet_index):
    # Connecting with Google Client
    scope = ['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive']
    creds = Credentials.from_service_account_file("data-analytics-key.json", scopes=scope)
    client = gspread.authorize(creds)                                            

    # Opening google sheet
    google_sh = client.open(sheet_name)
    sheet = google_sh.get_worksheet(worksheet_index)

    # write to dataframe
    sheet.clear()
    set_with_dataframe(worksheet=sheet, dataframe=df, include_index=False,
    include_column_header=True, resize=True)
    
    return print('Conversion complete')


# clear google sheet
def clean_sheet(df,sheet_name, worksheet_index):
    # Connecting with Google Client
    scope = ['https://www.googleapis.com/auth/spreadsheets',
            'https://www.googleapis.com/auth/drive']
    creds = Credentials.from_service_account_file("data-analytics-key.json", scopes=scope)
    client = gspread.authorize(creds)                                            

    # Opening google sheet
    google_sh = client.open(sheet_name)
    sheet = google_sh.get_worksheet(worksheet_index)
    sheet.clear()
   
    return print('Cleaning Complete')


## 1. Data Frames

### 1.1 Pool Info

In [24]:
df_info = querying_pagination(pool_info)
display(df_info)

,pool_name,pool_address,creation_date,token0_address,token0_symbol,token1_address,token1_symbol,fee_percent
0,UNI-WETH 3000 60,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801,2021-05-04,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,0.30
1,DAI-USDC 500 10,0x6c6bc977e13df9b0de53b251522280bb72383700,2021-05-04,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,0.05
2,USDC-WETH 10000 200,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,2021-05-04,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,1.00
3,WBTC-WETH 3000 60,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,2021-05-04,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,0.30
4,DAI-WETH 3000 60,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,2021-05-04,0x6b175474e89094c44da98b954eedeac495271d0f,DAI,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,0.30
...,...,...,...,...,...,...,...,...
8559,cbETH-WETH 100 1,0x177622e79acece98c39f6e12fa78ac7fc8a8bf62,2022-10-07,0xbe9895146f7af43049ca1c1ae358b0541ea49704,cbETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,0.01
8560,USDC-BUSC 3000 60,0xdc12cf3d73bc2141bf0179729b3b7fe98dac6465,2022-10-07,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,0xee31e8597d972cfd136042719bdbc8b24020f0d0,BUSC,0.30
8561,PZEN-USDC 500 10,0x24f4d82fc847b808ff413084070a1df8cdc11338,2022-10-07,0x5a24d7129b6f3fcad2220296df28911880ad22b0,PZEN,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,0.05
8562,UNI-EHIVE 10000 200,0x59c4708e800cfe7e54257aa9460fe91f3dc10e9d,2022-10-07,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI,0x4ae2cd1f5b8806a973953b76f9ce6d5fab9cdcfd,EHIVE,1.00


In [ ]:
df_to_sheet(df_info,"Uniswap Analyzer Dataset", 0)

### 1.2 Swap Volume

In [76]:
df_swap = querying_pagination(swap_vol)
df_swap.date = df_swap.date.apply(lambda x: dt.datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))
display(df_swap)

,date,pool_name,swap_count,amount_usd,fee_amount_usd
0,2022-04-12,USDC-SCIFI 3000 60,1,1.528423e+04,45.852690
1,2022-04-12,DAI-WETH 500 10,869,2.932858e+07,14664.289968
2,2022-04-12,CPD-WETH 3000 60,2,9.677500e+02,2.903250
3,2022-04-12,FPI-WETH 10000 200,28,3.308226e+05,3308.225700
4,2022-04-12,WETH-MFT 3000 60,20,8.076188e+04,242.285640
...,...,...,...,...,...
71157,2022-05-13,BAO-WETH 10000 200,3,7.057400e+02,7.057400
71158,2022-05-13,erowan-USDT 3000 60,9,5.483035e+03,16.449105
71159,2022-05-13,rETH-WETH 3000 60,20,3.830339e+05,1149.101760
71160,2022-05-13,STC-WETH 10000 200,5,1.009295e+03,10.092950


In [77]:
df_to_sheet(df_swap,"Uniswap Analyzer Dataset - Swap Vol", 0 )

Conversion complete


### 1.4 TVL Growth

In [93]:
df_tvl = querying_pagination(tvl_growth)
df_tvl.date = df_tvl.date.apply(lambda x: dt.datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))
display(df_tvl)

,date,pool_name,l_provider_type,net_amount_usd,unique_l_providers
0,2022-04-10,$TRDL-VBTC 3000 60,Whale,-1.477500e+04,1
1,2022-04-10,1INCH-USDC 10000 200,Common User,3.264765e+04,1
2,2022-04-10,1INCH-WETH 3000 60,Common User,-3.136800e+03,1
3,2022-04-10,20-WETH 10000 200,Common User,-4.917050e+01,1
4,2022-04-10,ALBT-WETH 10000 200,Whale,2.725916e+03,1
...,...,...,...,...,...
45909,2022-10-10,cbETH-WETH 3000 60,Whale,-6.902402e+04,2
45910,2022-10-10,cbETH-WETH 500 10,Common User,4.106872e+01,1
45911,2022-10-10,cbETH-WETH 500 10,Whale,1.262487e+06,2
45912,2022-10-10,gOHM-WETH 10000 200,Whale,7.408463e+02,1


In [94]:
df_to_sheet(df_tvl,"Uniswap Analyzer Dataset - TVL",0)

Conversion complete


### 1.4 TVL Overtime

In [96]:
df_over_tvl = querying_pagination(tvl_overtime)
df_over_tvl.date = df_over_tvl.date.apply(lambda x: dt.datetime.strptime(x.split(" ")[0], "%Y-%m-%d"))
display(df_over_tvl)

,date,pool_address,pool_name,token0_balance_adjusted,token1_balance_adjusted,tvl_usd,price_0_1,price_1_0,row_number
0,2022-10-09,0x0001fcbba8eb491c3ccfeddc5a5caba1a98c4c28,BCZ-WETH 10000 200,0.000000,0.000000,NaN,1.000199e+04,9.998009e-05,1
1,2022-10-09,0x0002e63328169d7feea121f1e32e4f620abf0352,wNXM-ICHI 3000 60,827.662379,3336.965012,31567.413276,3.364696e-01,2.972037e+00,1
2,2022-10-09,0x0003c9d75e102b8a9fe77280ca410e9135beb150,TRANCHE-bWBTC-A-USDT 10000 200,0.000000,0.000000,NaN,2.939251e-41,3.402228e+40,1
3,2022-10-09,0x000ea4a83acefdd62b1b43e9ccc281f442651520,BUSD-WETH 3000 60,0.000000,0.000000,0.000000,3.625292e+03,2.758398e-04,1
4,2022-10-09,0x000f0c0b0b791e855dcc5ad6501c7529dea882e0,WETH-DOGGE 3000 60,0.000000,0.000000,NaN,1.000098e+07,9.999016e-08,1
...,...,...,...,...,...,...,...,...,...
8579,2022-10-09,0xffae7370844307306672ced610936c149203dff2,dfxSGD-XSGD 100 1,0.000000,0.000000,NaN,1.000000e-12,1.000000e+12,1
8580,2022-10-09,0xffc9dc8e80049373ac068e6770ae519ef2c47e3e,USDT-FB 3000 60,0.000000,0.000000,NaN,1.001504e-02,9.984981e+01,1
8581,2022-10-09,0xffd93a85dddaba3096331c8e4d9c9783a24eed73,HLNK-USDC 3000 60,0.000000,0.000000,NaN,1.741368e+02,5.742612e-03,1
8582,2022-10-09,0xffe6008fcf63732ec8f666a91bf32e9be4eb7d56,SOX-WETH 3000 60,0.000000,0.000000,0.000000,4.112080e+01,2.431859e-02,1


In [97]:
df_to_sheet(df_over_tvl,"Uniswap Analyzer Dataset - TVL Overtime",0)

Conversion complete
